In [8]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout



In [9]:
# Load and preprocess data
data1 = pd.read_csv(r"C:\Users\sheet\Dataset\data_Q2_2022\dataq2.csv",encoding='utf-8',header=0)
filter_col = [col for col in data1 if col.endswith('_raw')]
data = data1.drop(columns= filter_col)



In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split



# Get the unique combinations of serial_number and model
unique_combinations = data[['serial_number', 'model']].drop_duplicates()

# Split the unique combinations into training and testing sets
train_combinations, test_combinations = train_test_split(unique_combinations, test_size=0.3, random_state=42)

# Filter the original dataset by the training and testing combinations
train = pd.merge(data, train_combinations, on=['serial_number', 'model'])
test = pd.merge(data, test_combinations, on=['serial_number', 'model'])

# Check that each unique combination of serial number and model is not present in both sets
assert len(set(train_combinations.index) & set(test_combinations.index)) == 0


In [19]:
# Selected features(using Boruta Feature selection technique)
input_cols = ['smart_1_normalized', 'smart_5_normalized', 'smart_9_normalized', 'smart_12_normalized', 'smart_187_normalized', 'smart_194_normalized', 'smart_197_normalized', 'smart_198_normalized','failure']


In [16]:
train_data = train[input_cols]
test_data =  test[input_cols]

In [17]:
# Define input and output sequences for LSTM network
seq_length = 30  # Length of each input sequence
X_train = []
y_train = []
for i in range(len(train_data) - seq_length - 1):
    X_train.append(train_data.iloc[i:i+seq_length].values)
    y_train.append(train_data.iloc[i+seq_length]['failure'])
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = []
y_test = []
for i in range(len(test_data) - seq_length - 1):
    X_test.append(test_data.iloc[i:i+seq_length].values)
    y_test.append(test_data.iloc[i+seq_length]['failure'])
X_test = np.array(X_test)
y_test = np.array(y_test)




In [18]:
# Build LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [20]:
# Train LSTM model
model.fit(X_train, y_train, epochs=50, batch_size=32)



Epoch 1/50
2986/2986 [==============================] - 463s 152ms/step - loss: nan - accuracy: 0.9893
Epoch 2/50
2986/2986 [==============================] - 3409s 1s/step - loss: nan - accuracy: 0.9893
Epoch 3/50
2986/2986 [==============================] - 208s 70ms/step - loss: nan - accuracy: 0.9893
Epoch 4/50
2986/2986 [==============================] - 214s 72ms/step - loss: nan - accuracy: 0.9893
Epoch 5/50
2986/2986 [==============================] - 202s 68ms/step - loss: nan - accuracy: 0.9893
Epoch 6/50
2986/2986 [==============================] - 205s 69ms/step - loss: nan - accuracy: 0.9893
Epoch 7/50
2986/2986 [==============================] - 205s 69ms/step - loss: nan - accuracy: 0.9893
Epoch 8/50
2986/2986 [==============================] - 221s 74ms/step - loss: nan - accuracy: 0.9893
Epoch 9/50
2986/2986 [==============================] - 208s 70ms/step - loss: nan - accuracy: 0.9893
Epoch 10/50
2986/2986 [==============================] - 212s 71ms/step - loss: na

In [21]:
# Evaluate LSTM model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 98.95
